In [1]:
# install syft if we are on google colab
import sys, os; os.system("pip install --pre syft") if "google.colab" in sys.modules else ""
import syft as sy
print(f"You're running syft version: {sy.__version__}")

You're running syft version: 0.7.0-beta.18


In [2]:
domains = {} # our logged in domain clients
domain_credentials = set() # our set of domain credentials

In [3]:
# enter the dict given from the data owner
def add_credentials(credentials_dict_list):
    for credentials_dict in credentials_dict_list:
        fs = frozenset(credentials_dict.items())
        domain_credentials.add(fs)

In [4]:
add_credentials(
    [
        {'url': '20.237.223.53', 'name': 'Samantha Carter', 'email': 'sam@sg1.net', 'password': 'stargate', 'dataset_name': 'MedNIST Data 1/10'},
        {'url': '20.237.223.58', 'name': 'Samantha Carter', 'email': 'sam@sg1.net', 'password': 'stargate', 'dataset_name': 'MedNIST Data 5/10'},
        #{'url': '20.237.223.187', 'name': 'Samantha Carter', 'email': 'sam@sg1.net', 'password': 'stargate', 'dataset_name': 'MedNIST Data 10/10'}
        #{'url': '20.237.223.58', 'name': 'Samantha Carter', 'email': 'info@openmined.org', 'password': 'changethis', 'dataset_name': 'MedNIST Data 5/10'},
    ]   
)

In [5]:
domain_credentials

{frozenset({('dataset_name', 'MedNIST Data 5/10'),
            ('email', 'sam@sg1.net'),
            ('name', 'Samantha Carter'),
            ('password', 'stargate'),
            ('url', '20.237.223.58')}),
 frozenset({('dataset_name', 'MedNIST Data 1/10'),
            ('email', 'sam@sg1.net'),
            ('name', 'Samantha Carter'),
            ('password', 'stargate'),
            ('url', '20.237.223.53')})}

In [6]:
def login_to_domains(force: bool = False):
    for fd in domain_credentials:
        credentials = dict(fd)
        if credentials["url"] not in domains or force:
            try:
                details = credentials.copy()
                del details["name"]
                del details["dataset_name"]
                client = sy.login(**details)
                domains[credentials["url"]] = client
            except Exception as e:
                print(e)

In [7]:
login_to_domains()

Connecting to 20.237.223.58... done! 	 Logging into clever_russell... done!
Connecting to 20.237.223.53... done! 	 Logging into strange_sutskever... done!


In [8]:
#domain_client = domains['20.237.223.58']

In [9]:
pixels, labels = None, None

for ip, domain_client in domains.items():
    data = domain_client.datasets[0]
    if pixels is None:
        pixels = data["images"]
        labels = data["labels"]
        pixels.public_shape = (5895, 64, 64)
        labels.public_shape = (5895,)
    else:
        _images = data["images"]
        _labels = data["labels"]
        _images.public_shape = (5895, 64, 64)
        _labels.public_shape = (5895,)
        pixels = pixels.concatenate(_images, )
        labels = labels.concatenate(_labels)

[2022-05-11T07:58:16.413480+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.440526+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.464718+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.483536+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.684981+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.707182+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.731108+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'
[2022-05-11T07:58:16.753517+0000][CRITICAL][logger]][32364] 'Class' object has no attribute 'generate_przs'


In [10]:
pixels, labels

(
 
 MPCTensor.shape=(11790, 64, 64)
 	 .child[0] = <TensorPointer -> clever_russell:d86bcffc8069a2726fe96a2205eec9e1>
 	 .child[1] = <TensorPointer -> strange_sutskever:0c5825ec366b63856748301a8c8fe926>,
 
 
 MPCTensor.shape=(11790,)
 	 .child[0] = <TensorPointer -> clever_russell:28609c5e32e1dc991a1904965fe5225e>
 	 .child[1] = <TensorPointer -> strange_sutskever:874b93ded2a6591a1ff8ca0913bee409>)

In [10]:
!pip install sklearn

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from tqdm.notebook import tqdm

In [12]:
X_train = pixels.T

In [13]:
X_train.get(proxy_only=True).shape

**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.


(4096, 5895)

In [202]:
# m, n = 5895, 4096
# split = 0.8
# idx = round(split * n)

# Y_dev = labels[0:idx]
# X_dev = pixels[0:idx]
# X_dev = X_dev / 255.

# Y_train = labels[idx:]
# X_train = pixels[idx:]
# X_train = X_train / 255.
X_train = pixels.T
X_train = X_train / 255.0
Y_train = labels / 255.

In [206]:
X_train.get(proxy_only=True).shape
Y_train.get(proxy_only=True).shape

**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.
**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.


(5895,)

In [207]:
def init_params():
    W1 = np.random.rand(6, 4096) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    print("W1", W1.shape)
    print("X", X.get(proxy_only=True).shape)
    Z1 = W1@(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2@(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2@(A1.T)
    db2 = 1 / m * dZ2.sum()
    dZ1 = W2.T@(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1@(X.T)
    db1 = 1 / m * dZ1.sum()
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [209]:
def get_predictions(A2):
    return np.argmax(A2)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 40 == 0:
            print("Predicition will be availabe in the next iteration.....Thank you for your support.")
            # predictions = get_predictions(A2)
            # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [211]:
print(X_train.get(proxy_only=True).shape)
print(Y_train.get(proxy_only=True).shape)

**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.
(4096, 5895)
**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.
(5895,)


In [212]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

[INFO]: Starting training!



  0%|          | 0/1 [00:00<?, ?it/s]

W1 (6, 4096)
**Warning**: Fetching proxy_only will not delete the real object
**Warning**: Proxy data class does not exist for this object. Fetching the real data.
X (4096, 5895)
{'request_block': True, 'timeout_secs': 25, 'delete_obj': False}


[2022-05-11T07:47:46.179144+0000][CRITICAL][logger]][25341] UnknownPrivateException has been triggered.
[2022-05-11T07:47:46.180319+0000][CRITICAL][logger]][25341] Request to access data length rejected.


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [179]:
W1, b1, W2, b2 = init_params()

In [214]:
x = W1@X_train

[2022-05-11T07:48:31.715898+0000][CRITICAL][logger]][25341] UnknownPrivateException has been triggered.
[2022-05-11T07:48:31.717206+0000][CRITICAL][logger]][25341] Request to access data length rejected.


{'request_block': True, 'timeout_secs': 25, 'delete_obj': False}


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [215]:
x = X_train.get_copy()

In [ ]:
x.child.child.child

In [ ]:
W1@x